In [1]:
import pandas as pd
import os 
import tarfile 
import urllib
import numpy as np
 
    
HOUSING_PATH = "C:\margo\ML\Geron\git_example\handson-ml2\datasets\housing"
    
def load_housing_data(housing_path=HOUSING_PATH): 
    csv_path = os.path.join(housing_path, "housing.csv")     
    return pd.read_csv(csv_path)

In [2]:
housing = load_housing_data(HOUSING_PATH)
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [3]:
def split_train_set(data, test_ratio):
        shuffled_indices = np.random.permutation(len(data))
        test_set_size = int(len(data) * test_ratio)
        test_indices = shuffled_indices[:test_set_size]
        train_indices = shuffled_indices[test_set_size:]     
        return data.iloc[train_indices], data.iloc[test_indices]

In [4]:
train_set, test_set = split_train_set(housing, 0.2)

In [5]:
print("train set lenght is:" + str(len(train_set)) + ";\ntest set lenght is: " +str(len(test_set)))

train set lenght is:16512;
test set lenght is: 4128


In [6]:
from sklearn.model_selection import StratifiedShuffleSplit 

housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6.0, 7.5, 9.0, np.inf],
                               labels=[1, 2, 3, 4, 5, 6, 7])

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42) 
for train_index, test_index in split.split(housing, housing["income_cat"]):     
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]
    

for set_ in (strat_train_set, strat_test_set):     
    set_.drop("income_cat", axis=1, inplace=True)

In [7]:
housing = strat_train_set.copy()

In [8]:
housing["rooms_per_household"] = housing["total_rooms"] / housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"] 
housing["population_per_household"]=housing["population"] / housing["households"]

In [9]:
housing = strat_train_set.drop("median_house_value", axis=1) 
housing_labels = strat_train_set["median_house_value"].copy()

In [10]:
from sklearn.impute import SimpleImputer 

housing_num = housing.drop("ocean_proximity", axis=1)
#imputer = SimpleImputer(strategy="median")
#imputer.fit(housing_num)

In [11]:
from sklearn.base import BaseEstimator, TransformerMixin 
 
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6 

In [12]:
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):     
    def __init__(self, add_bedrooms_per_room = True): # no * args or ** kargs
         self.add_bedrooms_per_room = add_bedrooms_per_room     
    def fit(self, X, y=None):         
        return self  # n o t h i n g e l s e t o d o
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:             
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room] 
 
        else:             
            return np.c_[X, rooms_per_household, population_per_household] 


In [13]:
#attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
#housing_extra_attribs = attr_adder.transform(housing.values)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler 
 
num_pipeline = Pipeline([         
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()), 
        ('std_scaler', StandardScaler()),     
    ]) 
 
housing_num_tr = num_pipeline.fit_transform(housing_num)

In [15]:
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import OneHotEncoder 
    
num_attribs = list(housing_num) 
cat_attribs = ["ocean_proximity"] 
 
full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),     
    ]) 
 
housing_prepared = full_pipeline.fit_transform(housing)

In [16]:
from sklearn.linear_model import LinearRegression 
 
lin_reg = LinearRegression() 
lin_reg.fit(housing_prepared, housing_labels)

LinearRegression()

In [17]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data) 
print("Predictions:", lin_reg.predict(some_data_prepared)) 

Predictions: [ 85567.3451932  471965.3878167  151916.14768674 186477.93483214
 244358.19381986]


In [18]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

svr = SVR()
svr.fit(housing_prepared, housing_labels)
svr_housing_predictions = svr.predict(housing_prepared)
svr_mse = mean_squared_error(housing_labels, svr_housing_predictions) 
svr_rmse = np.sqrt(svr_mse) 
svr_rmse

118357.29776637343

In [19]:
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)
svr_rbf.fit(housing_prepared, housing_labels)
svr_rbf_housing_predictions = svr_rbf.predict(housing_prepared)
svr_rbf_mse = mean_squared_error(housing_labels, svr_rbf_housing_predictions) 
svr_rbf_rmse = np.sqrt(svr_rbf_mse) 
svr_rbf_rmse

95429.84992073712

In [20]:
svr_lin = SVR(kernel='linear', C=100, gamma='auto')
svr_lin.fit(housing_prepared, housing_labels)
svr_lin_housing_predictions = svr_lin.predict(housing_prepared)
svr_lin_mse = mean_squared_error(housing_labels, svr_lin_housing_predictions) 
svr_lin_rmse = np.sqrt(svr_lin_mse) 
svr_lin_rmse

71237.10538307506

In [21]:
svr_poly = SVR(kernel='poly', C=100, gamma='auto', degree=3, epsilon=.1,
               coef0=1)
svr_poly.fit(housing_prepared, housing_labels)
svr_poly_housing_predictions = svr_poly.predict(housing_prepared)
svr_poly_mse = mean_squared_error(housing_labels, svr_poly_housing_predictions) 
svr_poly_rmse = np.sqrt(svr_poly_mse) 
svr_poly_rmse

75031.18355631913

In [22]:
svr_lin = SVR(kernel='linear', C=10, gamma='auto')
svr_lin.fit(housing_prepared, housing_labels)
svr_lin_housing_predictions = svr_lin.predict(housing_prepared)
svr_lin_mse = mean_squared_error(housing_labels, svr_lin_housing_predictions) 
svr_lin_rmse = np.sqrt(svr_lin_mse) 
svr_lin_rmse

81913.70157811412

In [23]:
svr_lin = SVR(kernel='linear', C=500, gamma='auto')
svr_lin.fit(housing_prepared, housing_labels)
svr_lin_housing_predictions = svr_lin.predict(housing_prepared)
svr_lin_mse = mean_squared_error(housing_labels, svr_lin_housing_predictions) 
svr_lin_rmse = np.sqrt(svr_lin_mse) 
svr_lin_rmse

70361.13950302043

In [38]:
def SvrBuildUp(housing_prepared, housing_labels, kernel_type, C_param, gamma_param):
    #kernel_type='linear', C_param=100, gamma_param='auto')
    svr = SVR(kernel=kernel_type, C=C_param, gamma=gamma_param)
    svr.fit(housing_prepared, housing_labels)
    svr_housing_predictions = svr.predict(housing_prepared)
    svr_mse = mean_squared_error(housing_labels, svr_housing_predictions) 
    return np.sqrt(svr_mse) 

svr_lin_rmse = SvrBuildUp(housing_prepared, housing_labels, 'linear', 100, 'auto')
print(svr_lin_rmse)

In [ ]:
from sklearn.model_selection import GridSearchCV 
 
param_grid = [     {'C': [10, 100, 500], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
              { 'C': [80, 100, 130], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'gamma':['scale', 'auto'], 
               'degree':[1, 2, 4, 8, 10], 'epsilon':[0.01, 0.05 ,.1, .2, .5, .8,]}, ]
               #'coef0' :[0, 0.5, 1, 2]},   ]


svr_search = SVR()
 
grid_search = GridSearchCV(svr_search, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True) 

#SVR().get_params().keys()
grid_search.fit(housing_prepared, housing_labels)

In [57]:
grid_search.best_params

AttributeError: 'GridSearchCV' object has no attribute 'best_params'

In [60]:
SVR().get_params().keys()

dict_keys(['C', 'cache_size', 'coef0', 'degree', 'epsilon', 'gamma', 'kernel', 'max_iter', 'shrinking', 'tol', 'verbose'])

In [ ]:
param_rand = {'C': reciprocal(10, 5000), 'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}


svr_rand = SVR()
 
    
rand_search = RandomizedSearchCV(svm_rand, param_rand,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error',
                                verbose=2, random_state=28, return_train_score=True)    


#SVR().get_params().keys()
rand_search.fit(housing_prepared, housing_labels)
